<a href="https://colab.research.google.com/github/hugeclear/RAG_llama3.2-1b/blob/master/rag_llama3_2_1b.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Token is set: True


In [6]:
from google.colab import userdata
from huggingface_hub import login

# シークレットからトークンを取得してログイン
token = userdata.get('HUGGINGFACE_TOKEN')
login(token=token)

print("Authentication completed")

Authentication completed


In [10]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
import time

print("Starting simple test...")

# まずGPUメモリをクリア
if torch.cuda.is_available():
    torch.cuda.empty_cache()
    print("GPU memory cleared")

# トークナイザーの初期化
print("Loading tokenizer...")
tokenizer = AutoTokenizer.from_pretrained(
    "meta-llama/Llama-3.2-1b",
    trust_remote_code=True
)
tokenizer.pad_token = tokenizer.eos_token
print("Tokenizer loaded")

# モデルの初期化（パラメータを制限）
print("Loading model...")
model = AutoModelForCausalLM.from_pretrained(
    "meta-llama/Llama-3.2-1b",
    device_map="auto",
    torch_dtype=torch.float16,
    trust_remote_code=True,
    max_memory={0: "4GB"}  # メモリ使用量を制限
)
print("Model loaded")

# 単一の短いプロンプト
prompt = "Write one sentence about a cat:"

# 入力の処理
inputs = tokenizer(
    prompt,
    return_tensors="pt",
    padding=True,
    truncation=True,
    max_length=30  # 入力長を制限
).to(model.device)

print("Generating response...")
start_time = time.time()

# 厳格な制限付きで生成
outputs = model.generate(
    **inputs,
    max_length=50,      # 出力長を厳しく制限
    min_length=5,       # 最小長も設定
    temperature=0.5,    # より決定論的に
    top_p=0.9,
    do_sample=True,
    num_beams=1,        # ビーム探索なし
    pad_token_id=tokenizer.pad_token_id,
    eos_token_id=tokenizer.eos_token_id,
    early_stopping=True # 早期停止を有効化
)

end_time = time.time()
print(f"Generation took {end_time - start_time:.2f} seconds")

response = tokenizer.decode(outputs[0], skip_special_tokens=True)
print("\nPrompt:", prompt)
print("Response:", response)

# 後処理
if torch.cuda.is_available():
    torch.cuda.empty_cache()
print("Test completed")

Starting test sequence...

Step 1: Loading tokenizer...

Step 2: Loading model...

Model initialization successful! Running generation tests...

Test 1: 犬について一言で説明してください：

Generating response for: 犬について一言で説明してください：


Exception in thread Thread-12:
Traceback (most recent call last):
  File "/usr/lib/python3.10/threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.10/threading.py", line 1378, in run
    self.function(*self.args, **self.kwargs)
  File "<ipython-input-10-59ef2620cbc0>", line 77, in timeout_handler
Exception: Timeout



Generation timeout! Taking too long...
Generation took 208.34 seconds

Response:
犬について一言で説明してください：　犬を飼う人は誰？
The Japanese dog is a small, slender dog with a short, broad muzzle, a round head, and erect ears. The body is well muscled, with the forequarters being slightly longer than the hindquarters. Dogs are usually black, brown, or gray. They may also be white, cream, tan, yellow, blue, red, fawn, black-and-white, chocolate, cinnamon,



In [12]:
# 必要なパッケージのインストール
!pip install -q chromadb
!pip install -q langchain
!pip install -q langchain-community
!pip install -q sentence-transformers
!pip install -q faiss-cpu

# インポート
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
import chromadb
import os

class RAGBase:
    def __init__(self):
        # ChromaDBクライアントの初期化
        self.chroma_client = chromadb.Client()

        # テキストスプリッターの初期化（日本語対応）
        self.text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=300,
            chunk_overlap=50,
            length_function=len,
            separators=["\n\n", "\n", "。", "、", " ", ""]
        )

        print("Initializing embeddings model...")
        # 埋め込みモデルの初期化
        self.embeddings = HuggingFaceEmbeddings(
            model_name="intfloat/multilingual-e5-small",
            model_kwargs={'device': 'cpu'}
        )

        # Chromaベクトルストアの初期化
        self.vectorstore = Chroma(
            collection_name="test_collection",
            embedding_function=self.embeddings
        )
        print("Initialization complete!")

    def add_texts(self, texts: list[str]):
        """テキストをデータベースに追加"""
        try:
            # テキストの分割
            splits = self.text_splitter.split_text(" ".join(texts))

            # ベクトルストアに追加
            self.vectorstore.add_texts(splits)

            print(f"Added {len(splits)} text chunks to the database")
            return True

        except Exception as e:
            print(f"Error adding texts: {str(e)}")
            return False

    def search_similar(self, query: str, k: int = 3):
        """類似テキストの検索"""
        try:
            # 類似度検索
            results = self.vectorstore.similarity_search(
                query,
                k=k
            )

            return results

        except Exception as e:
            print(f"Error during search: {str(e)}")
            return None

# テスト実行
def test_rag_system():
    print("Starting RAG system test...")
    rag = RAGBase()

    # テストデータ
    test_texts = [
        """
        機械学習は、コンピュータシステムが明示的なプログラミングなしでタスクを実行できるようにする技術です。
        データから学習し、パターンを見つけ、予測や判断を行います。
        教師あり学習、教師なし学習、強化学習などの手法があります。
        """,
        """
        深層学習は機械学習の一種で、人工ニューラルネットワークを使用します。
        複数の層を重ねることで、複雑なパターンを学習できます。
        画像認識や自然言語処理で高い性能を発揮します。
        """
    ]

    print("\nAdding test texts...")
    rag.add_texts(test_texts)

    print("\nTesting search functionality...")
    query = "ニューラルネットワークについて教えてください"
    results = rag.search_similar(query)

    print("\nSearch results:")
    if results:
        for doc in results:
            print("-" * 50)
            print(doc.page_content)

# テスト実行
test_rag_system()

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 32.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.3/409.3 kB 25.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 52.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 21.6 MB/s eta 0:00:00
Starting RAG system test...
Initializing embeddings model...


<ipython-input-12-9e8488ff771c>:30: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  self.embeddings = HuggingFaceEmbeddings(


modules.json:   0%|          | 0.00/387 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/498k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/167 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/200 [00:00<?, ?B/s]

<ipython-input-12-9e8488ff771c>:36: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  self.vectorstore = Chroma(


Initialization complete!

Adding test texts...


Added 1 text chunks to the database

Testing search functionality...

Search results:
--------------------------------------------------
機械学習は、コンピュータシステムが明示的なプログラミングなしでタスクを実行できるようにする技術です。
        データから学習し、パターンを見つけ、予測や判断を行います。
        教師あり学習、教師なし学習、強化学習などの手法があります。
         
        深層学習は機械学習の一種で、人工ニューラルネットワークを使用します。
        複数の層を重ねることで、複雑なパターンを学習できます。
        画像認識や自然言語処理で高い性能を発揮します。
